In [1]:
pip install torch

In [2]:
!pip install transformers
!pip install datasets

In [3]:
!pip install --upgrade transformers accelerate

In [4]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/ML-Project/PII-DATA'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import os
from pathlib import Path

import pandas as pd

train= pd.read_json('/content/drive/My Drive/ML-Project/PII-DATA/train.json')

test= pd.read_json('/content/drive/My Drive/ML-Project/PII-DATA/test.json')


In [6]:
type(train['labels'][0])

list

In [7]:
import pandas as pd

# Extract all labels from the DataFrame into a single list
all_labels = [label for sublist in train['labels'] for label in sublist]

# Get the unique labels as a set
unique_labels = set(all_labels)

# Calculate the number of unique labels
num_labels = len(unique_labels)

print(f"Number of unique labels: {num_labels}")


Number of unique labels: 13


In [8]:
unique_labels

{'B-EMAIL',
 'B-ID_NUM',
 'B-NAME_STUDENT',
 'B-PHONE_NUM',
 'B-STREET_ADDRESS',
 'B-URL_PERSONAL',
 'B-USERNAME',
 'I-ID_NUM',
 'I-NAME_STUDENT',
 'I-PHONE_NUM',
 'I-STREET_ADDRESS',
 'I-URL_PERSONAL',
 'O'}

In [9]:
train.head()

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."


In [10]:
label2id = {label: id for id, label in enumerate(unique_labels)}

In [11]:
label2id

{'B-STREET_ADDRESS': 0,
 'B-PHONE_NUM': 1,
 'B-EMAIL': 2,
 'B-ID_NUM': 3,
 'B-URL_PERSONAL': 4,
 'B-NAME_STUDENT': 5,
 'I-NAME_STUDENT': 6,
 'B-USERNAME': 7,
 'I-ID_NUM': 8,
 'O': 9,
 'I-PHONE_NUM': 10,
 'I-STREET_ADDRESS': 11,
 'I-URL_PERSONAL': 12}

In [12]:
def encode_labels(label_list):
    return [label2id[label] for label in label_list]

# Apply the encoding function to the 'labels' column
train['encoded_labels'] = train['labels'].apply(encode_labels)

In [13]:
train.head()

,document,full_text,tokens,trailing_whitespace,labels,encoded_labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-...","[9, 9, 9, 9, 9, 9, 9, 9, 9, 5, 6, 9, 9, 9, 9, ..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O...","[5, 6, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, ..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O...","[9, 9, 9, 9, 5, 6, 9, 9, 9, 9, 9, 9, 9, 9, 9, ..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT...","[9, 9, 9, 9, 9, 5, 6, 9, 9, 9, 9, 9, 9, 9, 9, ..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST...","[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 5, 6, 9, ..."


In [14]:
texts = train['tokens'].tolist()
labels = train['encoded_labels'].tolist()

In [15]:
def check_alignment(sentences, labels):
    for sentence, label_seq in zip(sentences, labels):
        if len(sentence) != len(label_seq):
            return False
    return True


# Check if each token in all sentences has a corresponding label
alignment_ok = check_alignment(texts, labels)
print(alignment_ok)  # This should print True if each token has a corresponding label


True


In [20]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForTokenClassification, Trainer, TrainingArguments



tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


# Tokenize the input texts
encodings = tokenizer(texts, truncation=True, padding=True, return_tensors="pt",max_length=512,  is_split_into_words=True)



class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)



# Initialize the model
model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)


training_args = TrainingArguments(
    output_dir='/content/drive/My Drive/ML-Project/PII-DATA/model_output',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    logging_dir='/content/drive/My Drive/ML-Project/PII-DATA/logs',
    logging_steps=10
)


train_dataset = CustomDataset(encodings, labels)



# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

# Train the model
trainer.train()

<ipython-input-20-a7ec38ad0739>:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


RuntimeError: stack expects each tensor to be equal size, but got [825] at entry 0 and [886] at entry 1